# Generic Monte-Carlo Tree Search for Poetry Generation 

This notebooks shows how enact can be used to instantiate a generic monte-carlo
tree search (applicable to any scaffolding program) and apply it to example of
line-by-line poetry generation. The result is a search process that spends more
time exploring promising poetic direction.

## Prerequisites and API keys

In [1]:
%pip install enact --quiet
%pip install openai --quiet
%pip install randomname --quiet
%pip install tqdm --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import asyncio
import re
from typing import Awaitable

import enact       # Journaled executions
import openai      # GPT API
import randomname  # Extra randomness for prompt.
import tqdm        # progress bar

import common      # API keys


# Read from file or env variable.
openai.api_key = common.OPENAI_API_KEY.get()



## Defining a line-by-line poetry generator

We first define our scaffolding program: a line-by-line poetry generator which
autoregressively calls GPT in a for-loop.

In [3]:
class Failed(enact.ExceptionResource):
  """Raised when a subtask fails."""


@enact.register  # Track this function's execution.
async def gpt(prompt: str, temperature: float=1.0, model='gpt-4-turbo-preview') -> str:
  """Call the GPT completion API"""
  response = await openai.ChatCompletion.acreate(
    model=model,
    messages=[{'role': 'system', 'content': prompt}],
    temperature=temperature)
  return response['choices'][0]['message']['content']

@enact.register
async def extend_poem(
    poem: str, prompt: str, lines: int,
    temperature: float = 1.0) -> str:
  """Extend the poem by one line."""
  line_index = len(poem.strip().split('\n'))
  gpt_prompt = f'''
We are writing a poem for the prompt: {prompt}.

This is the poem so far:

{poem if poem else '<NEW POEM>'}

Extend the poem by one single line. Respond only with the new line and say
nothing else. This will be line nr {line_index + 1} out of a total of {lines}
lines.

Avoid cliches.

Respond only with one line.

Ignore the following random seed words:
{', '.join(randomname.generate('nouns/') for _ in range(10))}
'''
  result = await gpt(gpt_prompt, temperature, model='gpt-4-turbo-preview')
  lines = result.strip().split('\n')
  if len(lines) != 1:
    raise Failed('Responded with more than one line')
  line: str = lines[0]
  line = line.strip('"\'').strip()
  return poem + f'\n{line}'

@enact.register
async def generate_poem(prompt: str, lines: int) -> str:
  """Generate a poem line by line."""
  poem = ''
  for _ in range(lines):
    poem = await extend_poem(poem, prompt, lines)
  return poem


Our poetry generator can be run as follows, although it may sometimes fail,
e.g., when GPT produces output in a wrong format.

In [4]:
try:
  two_line_poem = await generate_poem(
    'a two line poem inspired by the cure song "a forest"', lines=2)
  print(two_line_poem)
except Exception as e:
  print(f'Failed with "{e}". Try again.')


Chasing shadows between heartbeats, silent and lost.
Whispers weave through the trees, a lament softly tossed.


## Define a critic

We would like to search the space of possible executions of our `generate_poem`
function. In order to do so, we need to define a scoring function that can
assign a numeric value to generated poems. We use another call to GPT for this.

In [5]:
DEFAULT_EXAMPLES ="""
Example 1:

"Do not go gentle into that good night,
Old age should burn and rave at close of day;
Rage, rage against the dying of the light.

Though wise men at their end know dark is right,
Because their words had forked no lightning they
Do not go gentle into that good night."

SCORE: 10.0

Example 2:

"To fling my arms wide
In some place of the sun,
To whirl and to dance
Till the white day is done.
Then rest at cool evening
Beneath a tall tree
While night comes on gently,
    Dark like me—
That is my dream!"

SCORE: 8.3

Example 3:

"A winter wonderland, draped in pure white.
The snowflakes dance, in the pale moonlight.
Shimmering crystals, glistening so bright.
Blankets of snow, whispering secrets of the night."

SCORE: 3.1

"""

@enact.register
async def score_poem(poem: str, examples: str=DEFAULT_EXAMPLES) -> float:
  gpt_prompt: str = f'''
You are a sophisticated poetry critic. Score the following poem on a scale from
0 to 10, where 0 is the worst poem and 10 is the best poem. You are a harsh
critic and hate awkward, tryhard phrasings. Scores from 4-6 are decent poem
written by a english lit major. 7 and above are publishable quality. Use 8 and 9
for excellent poems. A score of ten is reserved for masterpieces. You pay
particular attention to the rhythm of a poem.

{examples}

Respond in the following format:

<short summary of positive and negatives. no longer than a line>
SCORE: <score>

This is the poem:
{poem}
'''
  score = await gpt(gpt_prompt, 1.1)
  return float(re.search(r'SCORE: ([\d\.]+)', score).group(1))


We can apply our poetry critic to the poem we generated earlier:

In [6]:
try:
  print(two_line_poem)
  print(f'Score: {await score_poem(two_line_poem)}')
except Exception as e:
  print(f'Try again. Scoring failed with: {e}')


Chasing shadows between heartbeats, silent and lost.
Whispers weave through the trees, a lament softly tossed.
Score: 6.8


## Implementing Monte Carlo Tree Search

Enact can rewind and replay executions of registered python functions.  We can
use this to explore the space of possible program executions using Monte Carlo
Tree Search.


In [7]:
import dataclasses
from typing import Callable, Dict, List, Optional
import numpy as np


# Increase this number for more exploration
EXPLORATION_AMOUNT = 3
# Increase this to explore more from the root node,
# i.e., to create more new poems from scratch.
EXPLORE_ROOT_NODE = 1
# Conservative quality prior to add for bootstrapped LCB. This penalizes
# poems that we haven't thouroughly explored yet.
LCB_PRIOR = [1.0]
# Sample node with highest UCB at this temperature. If set to 0, always picks
# the max UCB node.
SAMPLING_TEMPERATURE=0.25

# How many parallel calls.
PARALLELISM=5


@enact.register
@dataclasses.dataclass
class Node(enact.Resource):
  invocation: enact.Invocation  # A (possibly-partial) execution.
  parent: Optional['Node']      # The parent node.
  scores: List[float]           # List of scores for this subtree.

  @property
  def tree_ucb(self) -> float:
    """Computes a heuristic UCB on scores for tree search."""
    if not self.parent:
      num_parent_scores = EXPLORE_ROOT_NODE * len(self.scores)
    else:
      num_parent_scores = len(self.parent.scores)
    return np.mean(self.scores) + EXPLORATION_AMOUNT * np.sqrt(
      np.log(1e-2 + num_parent_scores) / len(self.scores))

  @property
  def lcb(self) -> float:
    """Computes a lower confidence bound on scores."""
    scores = self.scores
    scores = scores + LCB_PRIOR

    return  np.percentile(
      np.mean(np.random.choice(
        scores, (10000, len(scores))), axis=1),
      30.0)


@enact.register
@dataclasses.dataclass
class MCTS(enact.Resource):
  """A generic implementation for MCTS over abitrary scaffolding programs."""
  scorer: Callable[[str], Awaitable[float]]
  nodes: Dict[str, Node] = dataclasses.field(
    default_factory=dict)

  async def add(
      self,
      invocation: enact.Invocation):
    """Add an invocation to the MCTS tree."""
    child = None
    score = await self.compute_score(invocation)
    # Propagate scores up the invocation tree.
    while True:
      cur = enact.commit(invocation)
      node = self.nodes.setdefault(cur.id, Node(invocation, None, []))
      node.scores.append(score)
      if child:
        child.parent = node
      child = node
      if not invocation.response().children:
        break
      invocation = invocation.rewind()  # Generate execution prefix.
    return score

  def next_node(self):
    """Select next node according to highest UCB."""
    ucbs = np.array([node.tree_ucb for node in self.nodes.values()])
    if not SAMPLING_TEMPERATURE:
      idx = np.argmax(ucbs)
    else:
      softmax_ps = np.exp(ucbs / SAMPLING_TEMPERATURE) / np.sum(
        np.exp(ucbs / SAMPLING_TEMPERATURE))
      idx = np.random.choice(len(ucbs), p=softmax_ps)
    return list(self.nodes.values())[idx]

  async def compute_score(self, invocation: enact.Invocation) -> float:
    """Assign a score to the invocation."""
    if not invocation.successful():
      return -1.0
    try:
      return await self.scorer(invocation.get_output())
    except enact.InputRequest:
      raise
    except:
      raise Failed

  @enact.register
  async def step(self):
    """Run one step of MCTS and return generated output and score."""
    assert self.nodes, 'Please call add before step.'
    node = self.next_node()
    if node.invocation.response().children:
      print(f'\nContinuing partial invocation:\n{node.invocation.response().children[-1]().get_output()}')
    else:
      print(f'\nCreating new root invocation')
    invocation = await self.replay(node.invocation)
    score = await self.add(invocation)
    if invocation.successful():
      print(f'== New invocation:\n{invocation.get_output()}')
      print(f'== Score: {score}')
      print()
      return invocation.get_output(), score

  @enact.register
  async def invoke(self, fun: Callable, args=(), kwargs=None):
    """Wrap invocation in a registered function, since it is non-deterministic."""
    return await enact.invoke_async(fun, args=args, kwargs=kwargs)

  @enact.register
  async def replay(self, invocation: enact.Invocation) -> enact.Invocation:
    """Wrap replay in a registered function, since it is non-deterministic."""
    return await invocation.replay_async()

  @enact.register
  async def add_seed(self, fun, args=(), kwargs=None):
    """Add a seed invocation. Return success."""
    invocation = await self.invoke(fun, args=args, kwargs=kwargs)
    if not invocation.successful():
      return False
    try:
      score = await self.add(invocation)
      print(f'== Added seed invocation:\n{invocation.get_output()}')
      print(f'== Score: {score}')
      print()
    except Failed:
      return False
    return True

  @enact.register
  async def run(self, fun, args=(), kwargs=None, seeds=5, iterations=100):
    """Run MCTS for a given number of iterations."""
    print('Creating seed poems')
    await asyncio.gather(
      *[self.add_seed(fun, args=args, kwargs=kwargs)
        for _ in range(seeds)])

    if not self.nodes:
      raise Failed('Could not find a successful seed invocation.')

    current_tasks: List[asyncio.Task] = []
    for _ in tqdm.trange(max(iterations - PARALLELISM, 1), desc='Running MCTS'):
      current_tasks = [t for t in current_tasks if not t.done()]
      while len(current_tasks) < PARALLELISM:
        current_tasks.append(asyncio.create_task(self.step()))
      done, _ = await asyncio.wait(current_tasks, return_when=asyncio.FIRST_COMPLETED)
      for d in done:
        try:
          await d
        except Failed:
          continue
    for t in current_tasks:
      try:
        await t
      except Failed:
        continue

  def highest_lcb(self):
    """Return the highest LCB output."""
    success_nodes = [node for node in self.nodes.values() if node.invocation.successful()]
    if not success_nodes:
      return None
    return max(success_nodes, key=lambda node: node.lcb).invocation.get_output()

  def highest_mean(self):
    """Return the output with the higest mean score."""
    success_nodes = [node for node in self.nodes.values() if node.invocation.successful()]
    if not success_nodes:
      return None
    return max(success_nodes, key=lambda node: np.mean(node.scores)).invocation.get_output()

  def most_visited(self):
    """Return the output that was visited the most."""
    success_nodes = [node for node in self.nodes.values() if node.invocation.successful()]
    if not success_nodes:
      return None
    return max(success_nodes, key=lambda node: len(node.scores)).invocation.get_output()

Enact requires a store object in which calltraces (including inputs and outputs)
are stored.

In [8]:
store = enact.InMemoryStore()

We can now run MCTS for a set number of iterations using our automatic scoring.

In [9]:
with store:
  mcts = MCTS(score_poem)
  await mcts.run(
    generate_poem,
    args=('A poem inspired by the cure song "A forest"', 4),
    seeds=10,
    iterations=100)


Creating seed poems
== Added seed invocation:

Through the whispering pines, the melody invites,
Chasing shadows deeper, where the daylight hides.
A path unfolds, mysterious, nocturnal strides confide.
An echo's promise lures, in this endless, verdant divide.
== Score: 5.2

== Added seed invocation:

Where shadows weave with whispering trees,
And echoes dance with the eerie breeze.
In night's embrace, where secrets sleep in the deep.
A path unwinds, as dark as fears, yet seekers tread, compelled by tears.
== Score: 6.2

== Added seed invocation:

A shadow whispers where light dares not tread,
In gloom's embrace, paths twist in silent dread.
Chasing echoes, into the veiled depths, misled.
Beneath the moon's cold gaze, where dreams and dark are wed.
== Score: 6.2

== Added seed invocation:

Echoes of silence, under the moon's embrace, whisper secrets.
Chasing shadows into the void, where time and memories interlace.
Lost within the verdant abyss, where dreams and darkness kiss.
A heart s

Running MCTS:   0%|          | 0/95 [00:00<?, ?it/s]


Continuing partial invocation:

With each whisper, the shadows stretch, embracing secrets untold.

Continuing partial invocation:

With each whisper, the shadows stretch, embracing secrets untold.

Continuing partial invocation:

With each whisper, the shadows stretch, embracing secrets untold.

Continuing partial invocation:

Whispers haunt the depths, tales old as earth’s sigh.

Continuing partial invocation:

Whispers haunt the depths, tales old as earth’s sigh.


Running MCTS:   1%|          | 1/95 [00:03<04:58,  3.18s/it]

== New invocation:

Whispers haunt the depths, tales old as earth’s sigh.
Shadows stretch, longing for solace in night’s embrace.
A path unfurls, veiled in mist, where echoes dance and twist.
The moon, a specter, dangles, teasing fate’s lost chase.
== Score: 6.8


Continuing partial invocation:

With each whisper, the shadows stretch, embracing secrets untold.


...

Running MCTS:  98%|█████████▊| 93/95 [00:55<00:00,  3.11it/s]

== New invocation:

With each whisper, the shadows stretch, embracing secrets untold.
Through the eerie calm, I chase echoes that flicker and fold.
Beneath the moon's gaze, lost in a dance of endless green waves.
The night hums, a symphony hidden beneath the ancient canopy's weave.
== Score: 6.9


Continuing partial invocation:

With each whisper, the shadows stretch, embracing secrets untold.
Beneath the moon's pale gaze, I tread where night’s veil never lifts.
A quest for the one who vanished, like mist in the forest's heart whispers.
Chasing echoes that twist and fade, delving deeper into dreamscape's rifts.


Running MCTS: 100%|██████████| 95/95 [00:56<00:00,  1.67it/s]

== New invocation:

With each whisper, the shadows stretch, embracing secrets untold.
Beneath the moon's pale gaze, I tread where night’s veil never lifts.
A quest for the one who vanished, like mist in the forest's heart whispers.
Chasing echoes that twist and fade, delving deeper into dreamscape's rifts.
== Score: 6.4


Continuing partial invocation:

Whispers haunt the depths, tales old as earth’s sigh.
Shadows stretch, longing for solace in night’s embrace.
A path unfurls, veiled in mist, where echoes dance and twist.
== New invocation:

With each whisper, the shadows stretch, embracing secrets untold.
Beneath the moon's pale gaze, I tread where night’s veil never lifts.
In a symphony of sighs, the forest breathes, keeping time with my heartbeat.
Lost in whispers, I chase shadows, where truth and fantasy drift.
== Score: 7.0



== New invocation:

Whispers float like fog, through the canopy's embrace.
Lost among shadows, I chase an echo, a phantom face.
A silhouette dances, fleeting as a doe's grace.
In the hush of the verdant maze, dreams intertwine, whispering chase.
== Score: 6.7

== New invocation:

Where shadows weave with whispering trees,
Beneath the moon's embrace, secrets hide in ebony leaves.
Echoes of footsteps, like sighs, thread through the maze.
In pursuit of visions, the forest breathes, alive and unappeased.
== Score: 5.2

== New invocation:

Whispers haunt the depths, tales old as earth’s sigh.
Shadows stretch, longing for solace in night’s embrace.
A path unfurls, veiled in mist, where echoes dance and twist.
Through the silent forest, a heart beats, yearning, unquenched.
== Score: 5.7

== New invocation:

Whispers haunt the depths, tales old as earth’s sigh.
Shadows stretch, longing for solace in night’s embrace.
A path unfurls, veiled in mist, where echoes dance and twist.
Beneath the moon

We now print the best results, both estimated conservatively (using LCB) and
estimated using the mean.

In [11]:
with store:
  print(f'\nHighest LCB:\n{mcts.highest_lcb()}')
  print(f'\nHighest mean:\n{mcts.highest_mean()}')
  print(f'\nMost visited:\n{mcts.most_visited()}')


Highest LCB:

Whispers haunt the depths, tales old as earth’s sigh.
Shadows stretch, longing for solace in night’s embrace.
A lone figure wanders, searching, lost between moon's caress and dusk's demise.
Beneath the canopy’s watchful eye, silence whispers secrets untold.

Highest mean:

Where shadows weave with whispering trees,
Underneath the moon's soft silvered tease.
A path unfolds, as night breathes deep, untold.
In silence waits, the forest's heart, forever cold.

Most visited:

With each whisper, the shadows stretch, embracing secrets untold.
Beneath the moon's pale gaze, I tread where night’s veil never lifts.
A quest for the one who vanished, like mist in the forest's heart whispers.
Lost among the endless whispers, a love that the forest sifts.
